# 🌾 Geração Sintética de Gramíneas de Pastos Brasileiros

Este notebook foca na geração e detecção específica de gramíneas forrageiras típicas dos pastos brasileiros.

---

## 📦 Instalação e Configuração

In [ ]:
# Instalar dependências específicas para gramíneas
!pip install controlnet-aux ultralytics xformers diffusers transformers --upgrade --quiet
!pip install segment-anything opencv-python-headless scikit-image --quiet

# Verificar instalação GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM disponível: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 🔧 Importações e Configuração

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import warnings
warnings.filterwarnings("ignore")

# Configurar device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando device: {device}")

# Configurar matplotlib para melhor visualização
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100

## 🤖 Carregamento do Modelo Stable Diffusion

In [ ]:
# Carregar modelo Stable Diffusion otimizado para natureza
model_id = "runwayml/stable-diffusion-v1-5"

print("Carregando modelo Stable Diffusion para geração de gramíneas...")
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False
).to(device)

# Usar scheduler mais rápido
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Otimização para Colab
if device == "cuda":
    try:
        pipe.enable_model_cpu_offload()
        print("CPU offload habilitado")
    except:
        print("CPU offload não disponível")

print("✅ Modelo carregado com sucesso!")

## 🌾 Definição de Gramíneas Brasileiras

Principais gramíneas forrageiras dos pastos brasileiros:
- **Brachiaria** (decumbens, brizantha, humidicola)
- **Panicum** (mombaça, tanzânia, massai)
- **Cynodon** (tifton, coast-cross)
- **Andropogon** (capim-andropógon)
- **Setaria** (capim-setária)

In [ ]:
# Prompts específicos para gramíneas brasileiras
grass_prompts = [
    # Brachiaria - principal gramínea dos pastos brasileiros
    "close up view of Brachiaria brizantha grass field, green forage grass, detailed grass texture, brazilian tropical pasture, natural lighting, high resolution",
    "Brachiaria decumbens grassland, tropical forage grass, green pasture field, detailed grass blades, sunny day, photorealistic",
    "aerial view of Brachiaria humidicola field, dense grass coverage, green forage, tropical pasture, natural texture, high quality",
    
    # Panicum - gramíneas de alto valor nutritivo
    "Panicum maximum mombaça grass, tall tropical grass, green forage field, detailed grass structure, natural lighting, realistic",
    "Panicum tanzânia grassland, tropical forage grass, green field, grass detail, sunny pasture, photographic quality",
    "Panicum massai grass field, compact tropical grass, green pasture, detailed vegetation, natural scenery",
    
    # Cynodon - gramíneas resistentes
    "Cynodon dactylon tifton grass, fine textured grass, green lawn, detailed grass blades, uniform coverage, high resolution",
    "coast-cross Cynodon grass field, tropical pasture grass, green coverage, detailed texture, natural lighting",
    
    # Gramíneas nativas do cerrado
    "Andropogon gayanus grass field, native brazilian grass, cerrado vegetation, natural grassland, green forage",
    "native brazilian grassland, mixed tropical grasses, natural pasture, green field, cerrado landscape",
    "Setaria grass field, tropical forage grass, green pasture, detailed grass texture, brazilian countryside",
    
    # Diferentes condições e estágios
    "young brazilian grass shoots, fresh green forage, new growth, detailed grass blades, natural lighting",
    "mature tropical grass field, established pasture, dense green coverage, brazilian forage grass",
    "mixed brazilian pasture grasses, biodiversity, tropical grassland, natural field, green vegetation",
    "drought resistant brazilian grass, dry season pasture, adapted tropical grass, resilient vegetation"
]

print(f"📝 Definidos {len(grass_prompts)} prompts específicos para gramíneas brasileiras")

# Prompt negativo otimizado para gramíneas
negative_prompt = (
    "animals, cattle, cows, sheep, horses, people, humans, buildings, fences, roads, "
    "artificial objects, weeds, flowers, trees, bushes, low quality, blurry, distorted, "
    "ugly, cartoon, painting, drawing, artificial"
)

## 🎨 Geração de Imagens de Gramíneas Específicas

In [ ]:
# Gerar imagens de gramíneas específicas
print(f"🌾 Gerando {len(grass_prompts)} imagens de gramíneas brasileiras...")
grass_images = []

for i, prompt in enumerate(grass_prompts):
    print(f"Gerando imagem {i+1}/{len(grass_prompts)}: {prompt[:50]}...")
    
    try:
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=30,  # Mais steps para melhor qualidade
            guidance_scale=8.0,      # Guidance mais alto para melhor aderência ao prompt
            width=512,
            height=512,
            generator=torch.Generator(device=device).manual_seed(100 + i)  # Seeds consistentes
        ).images[0]
        
        grass_images.append(image)
        
        # Progresso a cada 3 imagens
        if (i + 1) % 3 == 0:
            print(f"✅ {i+1} imagens de gramíneas concluídas")
            
    except Exception as e:
        print(f"❌ Erro ao gerar imagem {i+1}: {e}")
        # Adicionar imagem em branco para manter índices
        grass_images.append(Image.new('RGB', (512, 512), (255, 255, 255)))

print(f"🎉 {len([img for img in grass_images if img.size == (512, 512)])} imagens de gramíneas geradas com sucesso!")

## 📸 Visualização das Gramíneas Geradas

In [ ]:
# Exibir galeria de gramíneas
if grass_images:
    num_images = len(grass_images)
    cols = 4
    rows = (num_images + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
    fig.suptitle('🌾 Galeria de Gramíneas Brasileiras Sintéticas', fontsize=16, fontweight='bold')

    # Garantir que axes seja 2D
    if rows == 1:
        axes = axes.reshape(1, -1)
    if cols == 1:
        axes = axes.reshape(-1, 1)

    # Nomes das gramíneas para legendas
    grass_names = [
        "Brachiaria brizantha", "Brachiaria decumbens", "Brachiaria humidicola",
        "Panicum mombaça", "Panicum tanzânia", "Panicum massai",
        "Cynodon tifton", "Cynodon coast-cross",
        "Andropogon gayanus", "Gramíneas nativas", "Setaria",
        "Gramíneas jovens", "Gramíneas maduras", "Pastagem mista", "Gramíneas resistentes"
    ]

    for i in range(num_images):
        row = i // cols
        col = i % cols
        
        axes[row, col].imshow(grass_images[i])
        
        # Título com nome da gramínea
        title = grass_names[i] if i < len(grass_names) else f"Gramínea {i+1}"
        axes[row, col].set_title(title, fontsize=10, fontweight='bold')
        axes[row, col].axis('off')

    # Esconder eixos vazios
    for i in range(num_images, rows * cols):
        row = i // cols
        col = i % cols
        axes[row, col].axis('off')

    plt.tight_layout()
    plt.show()

    # Salvar imagens com nomes específicos
    print("\n💾 Salvando imagens de gramíneas...")
    for i, (image, name) in enumerate(zip(grass_images, grass_names)):
        if i < len(grass_names):
            filename = f"graminea_{name.lower().replace(' ', '_').replace('ã', 'a').replace('ç', 'c')}_{i+1:02d}.png"
        else:
            filename = f"graminea_{i+1:02d}.png"
        
        image.save(filename)
        print(f"✅ {filename}")

    print(f"\n🌾 {len(grass_images)} imagens de gramíneas brasileiras salvas!")

## 🔍 Detecção Avançada de Gramíneas com YOLO

In [ ]:
# Configurar YOLO especificamente para gramíneas
try:
    from ultralytics import YOLO
    import cv2
    import numpy as np
    
    print("🔍 Inicializando detecção de gramíneas com YOLO...")
    
    # Carregar modelo de segmentação (melhor para gramíneas)
    model = YOLO('yolov8n-seg.pt')
    
    # Função para filtrar classes relacionadas a plantas/gramíneas
    def is_vegetation_class(class_name, confidence):
        """Determina se a classe detectada é relacionada à vegetação"""
        vegetation_keywords = [
            'grass', 'plant', 'vegetation', 'field', 'lawn', 'meadow',
            'pasture', 'green', 'forage', 'leaf', 'bush', 'shrub'
        ]
        
        class_lower = class_name.lower()
        
        # Verificar palavras-chave específicas
        keyword_match = any(keyword in class_lower for keyword in vegetation_keywords)
        
        # Aceitar também classes com alta confiança (podem ser gramíneas não categorizadas)
        high_confidence = confidence > 0.7
        
        return keyword_match or high_confidence
    
    # Analisar algumas imagens de gramíneas
    if grass_images:
        print(f"Analisando {min(6, len(grass_images))} imagens de gramíneas...")
        
        grass_analysis_results = []
        
        for img_idx in range(min(6, len(grass_images))):
            print(f"\n🌾 Analisando gramínea {img_idx+1}...")
            
            # Executar detecção YOLO
            results = model(grass_images[img_idx], conf=0.25)  # Confiança mais baixa para captar mais vegetação
            result = results[0]
            
            # Analisar resultados
            analysis = {
                'image_idx': img_idx,
                'total_detections': 0,
                'vegetation_detections': 0,
                'vegetation_classes': [],
                'coverage_percentage': 0,
                'grass_mask': None
            }
            
            img_array = np.array(grass_images[img_idx])
            
            if result.masks is not None and len(result.masks) > 0:
                # Processar máscaras de segmentação
                masks = result.masks.data.cpu().numpy()
                classes = result.boxes.cls.cpu().numpy()
                confidences = result.boxes.conf.cpu().numpy()
                
                analysis['total_detections'] = len(masks)
                
                # Combinar máscaras de vegetação
                vegetation_mask = np.zeros(masks[0].shape)
                
                for mask, cls, conf in zip(masks, classes, confidences):
                    class_name = model.names[int(cls)]
                    confidence = float(conf)
                    
                    if is_vegetation_class(class_name, confidence):
                        vegetation_mask += mask
                        analysis['vegetation_detections'] += 1
                        analysis['vegetation_classes'].append((class_name, confidence))
                        print(f"  Detectado: {class_name} (confiança: {confidence:.2f})")
                
                # Normalizar máscara
                vegetation_mask = np.clip(vegetation_mask, 0, 1)
                analysis['grass_mask'] = vegetation_mask
                
                # Calcular cobertura
                total_pixels = vegetation_mask.shape[0] * vegetation_mask.shape[1]
                vegetation_pixels = np.sum(vegetation_mask > 0)
                analysis['coverage_percentage'] = (vegetation_pixels / total_pixels) * 100
                
                print(f"  Cobertura de gramíneas: {analysis['coverage_percentage']:.1f}%")
                
            else:
                print(f"  ⚠️ Nenhuma máscara de segmentação detectada")
            
            grass_analysis_results.append(analysis)
        
        print("\n✅ Análise YOLO de gramíneas concluída!")
        
    else:
        print("❌ Nenhuma imagem de gramínea disponível")
        
except ImportError as e:
    print(f"❌ YOLO não disponível: {e}")
    grass_analysis_results = []
except Exception as e:
    print(f"⚠️ Erro na análise YOLO: {e}")
    grass_analysis_results = []

## 📊 Visualização dos Resultados de Detecção

In [ ]:
# Visualizar resultados da detecção de gramíneas
if 'grass_analysis_results' in locals() and grass_analysis_results:
    print("📊 Visualizando resultados da detecção de gramíneas...")
    
    # Criar figura para mostrar análises
    valid_results = [r for r in grass_analysis_results if r['grass_mask'] is not None]
    
    if valid_results:
        num_results = len(valid_results)
        cols = 3
        rows = num_results
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
        fig.suptitle('🌾 Análise de Detecção de Gramíneas', fontsize=16, fontweight='bold')
        
        if rows == 1:
            axes = axes.reshape(1, -1)
        
        for i, analysis in enumerate(valid_results):
            img_idx = analysis['image_idx']
            img = grass_images[img_idx]
            mask = analysis['grass_mask']
            
            # Imagem original
            axes[i, 0].imshow(img)
            axes[i, 0].set_title(f"Gramínea {img_idx+1} - Original")
            axes[i, 0].axis('off')
            
            # Máscara de gramíneas
            axes[i, 1].imshow(mask, cmap='Greens', alpha=0.8)
            axes[i, 1].set_title(f"Máscara Detectada\n{analysis['coverage_percentage']:.1f}% cobertura")
            axes[i, 1].axis('off')
            
            # Overlay
            img_array = np.array(img)
            overlay = img_array.copy()
            
            # Criar overlay verde para áreas de gramínea
            green_overlay = np.zeros_like(img_array)
            green_overlay[:, :, 1] = mask * 255  # Canal verde
            
            # Combinar com imagem original
            result_overlay = cv2.addWeighted(img_array, 0.7, green_overlay, 0.3, 0)
            
            axes[i, 2].imshow(result_overlay)
            axes[i, 2].set_title(f"Gramíneas Destacadas\n{analysis['vegetation_detections']} detecções")
            axes[i, 2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Estatísticas resumidas
        print("\n📈 Estatísticas de Detecção:")
        avg_coverage = np.mean([r['coverage_percentage'] for r in valid_results])
        avg_detections = np.mean([r['vegetation_detections'] for r in valid_results])
        
        print(f"Cobertura média de gramíneas: {avg_coverage:.1f}%")
        print(f"Número médio de detecções por imagem: {avg_detections:.1f}")
        print(f"Total de imagens analisadas: {len(valid_results)}")
        
        # Classes mais detectadas
        all_classes = []
        for result in valid_results:
            all_classes.extend([cls[0] for cls in result['vegetation_classes']])
        
        if all_classes:
            from collections import Counter
            class_counts = Counter(all_classes)
            print(f"\nClasses mais detectadas:")
            for class_name, count in class_counts.most_common(5):
                print(f"  {class_name}: {count} vezes")
    
    else:
        print("❌ Nenhum resultado válido de detecção encontrado")
        
        # Mostrar estatísticas básicas mesmo sem detecções
        total_analyzed = len(grass_analysis_results)
        with_detections = len([r for r in grass_analysis_results if r['total_detections'] > 0])
        
        print(f"\n📊 Resumo da Análise:")
        print(f"Total de imagens analisadas: {total_analyzed}")
        print(f"Imagens com detecções: {with_detections}")
        print(f"Taxa de detecção: {(with_detections/total_analyzed)*100:.1f}%" if total_analyzed > 0 else "N/A")

else:
    print("❌ Análise YOLO não foi executada ou não há resultados disponíveis")

## 🌱 Segmentação Personalizada de Gramíneas por Cor

In [ ]:
# Segmentação avançada de gramíneas baseada em características visuais
def segment_grass_brazilian(image):
    """
    Segmentação específica para gramíneas brasileiras
    """
    img_array = np.array(image)
    
    # Converter para diferentes espaços de cor
    hsv = cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
    lab = cv2.cvtColor(img_array, cv2.COLOR_RGB2LAB)
    
    # Máscaras de cor para diferentes tipos de gramíneas brasileiras
    masks = []
    
    # 1. Gramíneas verdes vibrantes (Brachiaria jovem)
    lower_green1 = np.array([30, 50, 50])  # Verde vibrante
    upper_green1 = np.array([80, 255, 255])
    mask1 = cv2.inRange(hsv, lower_green1, upper_green1)
    masks.append(mask1)
    
    # 2. Gramíneas verde-amareladas (Panicum em crescimento)
    lower_green2 = np.array([20, 30, 40])  # Verde amarelado
    upper_green2 = np.array([60, 255, 200])
    mask2 = cv2.inRange(hsv, lower_green2, upper_green2)
    masks.append(mask2)
    
    # 3. Gramíneas secas/dormentes (época seca)
    lower_brown = np.array([10, 20, 30])   # Tons amarronzados
    upper_brown = np.array([30, 150, 150])
    mask3 = cv2.inRange(hsv, lower_brown, upper_brown)
    masks.append(mask3)
    
    # Combinar todas as máscaras
    combined_mask = np.zeros_like(mask1)
    for mask in masks:
        combined_mask = cv2.bitwise_or(combined_mask, mask)
    
    # Operações morfológicas para refinar
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    
    # Fechar buracos pequenos
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel_close)
    
    # Remover ruídos pequenos
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel_open)
    
    # Filtrar por área (remover regiões muito pequenas)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_mask = np.zeros_like(combined_mask)
    
    min_area = 200  # Área mínima para gramíneas
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area:
            cv2.fillPoly(filtered_mask, [contour], 255)
    
    return filtered_mask, masks

# Aplicar segmentação personalizada
if grass_images:
    print("🌱 Aplicando segmentação personalizada para gramíneas brasileiras...")
    
    # Analisar primeiras 4 imagens
    custom_analysis = []
    
    for i in range(min(4, len(grass_images))):
        print(f"Segmentando gramínea {i+1}...")
        
        grass_mask, individual_masks = segment_grass_brazilian(grass_images[i])
        
        # Calcular estatísticas
        total_pixels = grass_mask.shape[0] * grass_mask.shape[1]
        grass_pixels = np.sum(grass_mask > 0)
        coverage = (grass_pixels / total_pixels) * 100
        
        custom_analysis.append({
            'image_idx': i,
            'mask': grass_mask,
            'individual_masks': individual_masks,
            'coverage': coverage,
            'grass_pixels': grass_pixels,
            'total_pixels': total_pixels
        })
        
        print(f"  Cobertura detectada: {coverage:.1f}%")
    
    print("✅ Segmentação personalizada concluída!")

else:
    print("❌ Nenhuma imagem disponível para segmentação")
    custom_analysis = []

## 📊 Comparação: YOLO vs Segmentação Personalizada

In [ ]:
# Comparar resultados YOLO vs segmentação personalizada
if custom_analysis and 'grass_analysis_results' in locals():
    print("📊 Comparando métodos de detecção de gramíneas...")
    
    # Criar visualização comparativa
    num_compare = min(len(custom_analysis), 3)  # Comparar até 3 imagens
    
    fig, axes = plt.subplots(num_compare, 4, figsize=(20, 5 * num_compare))
    fig.suptitle('🔬 Comparação: YOLO vs Segmentação Personalizada', fontsize=16, fontweight='bold')
    
    if num_compare == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(num_compare):
        img_idx = custom_analysis[i]['image_idx']
        img = grass_images[img_idx]
        custom_mask = custom_analysis[i]['mask']
        custom_coverage = custom_analysis[i]['coverage']
        
        # Encontrar resultado YOLO correspondente
        yolo_result = None
        yolo_coverage = 0
        yolo_mask = None
        
        for result in grass_analysis_results:
            if result['image_idx'] == img_idx and result['grass_mask'] is not None:
                yolo_result = result
                yolo_coverage = result['coverage_percentage']
                yolo_mask = result['grass_mask']
                break
        
        # 1. Imagem original
        axes[i, 0].imshow(img)
        axes[i, 0].set_title(f"Gramínea {img_idx+1}\nOriginal")
        axes[i, 0].axis('off')
        
        # 2. YOLO
        if yolo_mask is not None:
            axes[i, 1].imshow(yolo_mask, cmap='Blues', alpha=0.8)
            axes[i, 1].set_title(f"YOLO\n{yolo_coverage:.1f}% cobertura")
        else:
            axes[i, 1].text(0.5, 0.5, 'YOLO\nSem detecção', 
                           ha='center', va='center', transform=axes[i, 1].transAxes)
            axes[i, 1].set_title("YOLO\n0% cobertura")
        axes[i, 1].axis('off')
        
        # 3. Segmentação personalizada
        axes[i, 2].imshow(custom_mask, cmap='Greens', alpha=0.8)
        axes[i, 2].set_title(f"Segmentação\nPersonalizada\n{custom_coverage:.1f}% cobertura")
        axes[i, 2].axis('off')
        
        # 4. Comparação lado a lado
        if yolo_mask is not None:
            # Criar imagem comparativa
            comparison = np.zeros((*custom_mask.shape, 3))
            comparison[:, :, 0] = yolo_mask * 255      # YOLO em vermelho
            comparison[:, :, 1] = custom_mask * 255    # Personalizada em verde
            # Sobreposição aparece em amarelo
            
            axes[i, 3].imshow(comparison.astype(np.uint8))
            axes[i, 3].set_title(f"Sobreposição\nAzul: YOLO\nVerde: Personalizada\nAmarelo: Ambos")
        else:
            axes[i, 3].imshow(custom_mask, cmap='Greens', alpha=0.8)
            axes[i, 3].set_title(f"Apenas\nSegmentação\nPersonalizada")
        axes[i, 3].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas comparativas
    print("\n📈 Estatísticas Comparativas:")
    
    custom_coverages = [analysis['coverage'] for analysis in custom_analysis]
    avg_custom = np.mean(custom_coverages)
    
    if grass_analysis_results:
        valid_yolo = [r for r in grass_analysis_results if r['grass_mask'] is not None]
        if valid_yolo:
            yolo_coverages = [r['coverage_percentage'] for r in valid_yolo]
            avg_yolo = np.mean(yolo_coverages)
            
            print(f"Cobertura média - YOLO: {avg_yolo:.1f}%")
            print(f"Cobertura média - Personalizada: {avg_custom:.1f}%")
            print(f"Diferença média: {abs(avg_custom - avg_yolo):.1f}%")
        else:
            print(f"Cobertura média - YOLO: 0% (sem detecções válidas)")
            print(f"Cobertura média - Personalizada: {avg_custom:.1f}%")
    else:
        print(f"Cobertura média - Personalizada: {avg_custom:.1f}%")
        print("YOLO não foi executado")

elif custom_analysis:
    print("📊 Mostrando apenas resultados da segmentação personalizada...")
    
    # Mostrar apenas resultados personalizados
    num_show = min(len(custom_analysis), 4)
    
    fig, axes = plt.subplots(2, num_show, figsize=(5 * num_show, 10))
    fig.suptitle('🌱 Segmentação Personalizada de Gramíneas', fontsize=16, fontweight='bold')
    
    if num_show == 1:
        axes = axes.reshape(-1, 1)
    
    for i in range(num_show):
        img_idx = custom_analysis[i]['image_idx']
        img = grass_images[img_idx]
        mask = custom_analysis[i]['mask']
        coverage = custom_analysis[i]['coverage']
        
        # Imagem original
        axes[0, i].imshow(img)
        axes[0, i].set_title(f"Gramínea {img_idx+1}")
        axes[0, i].axis('off')
        
        # Máscara
        axes[1, i].imshow(mask, cmap='Greens', alpha=0.8)
        axes[1, i].set_title(f"Segmentação\n{coverage:.1f}% cobertura")
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas
    avg_coverage = np.mean([a['coverage'] for a in custom_analysis])
    print(f"\n📊 Cobertura média detectada: {avg_coverage:.1f}%")

else:
    print("❌ Nenhuma análise disponível para comparação")

## 🌾 Análise de Qualidade das Gramíneas

In [ ]:
# Análise de qualidade específica para pastagens brasileiras
def analyze_pasture_quality(image, grass_mask, grass_type="Desconhecido"):
    """
    Análise de qualidade da pastagem baseada em características visuais
    """
    img_array = np.array(image)
    hsv = cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
    
    # Extrair regiões de gramínea
    grass_regions = img_array[grass_mask > 0]
    
    if len(grass_regions) == 0:
        return {
            'quality_score': 0,
            'quality_class': 'Sem gramíneas detectadas',
            'coverage': 0,
            'vigor_index': 0,
            'uniformity': 0
        }
    
    # 1. Cobertura
    total_pixels = grass_mask.shape[0] * grass_mask.shape[1]
    grass_pixels = np.sum(grass_mask > 0)
    coverage = (grass_pixels / total_pixels) * 100
    
    # 2. Índice de vigor (baseado na saturação e valor do verde)
    hsv_grass = hsv[grass_mask > 0]
    avg_saturation = np.mean(hsv_grass[:, 1])
    avg_value = np.mean(hsv_grass[:, 2])
    vigor_index = (avg_saturation + avg_value) / 2
    
    # 3. Uniformidade (baseada no desvio padrão da cor)
    color_std = np.mean([np.std(grass_regions[:, i]) for i in range(3)])
    uniformity = max(0, 100 - color_std)  # Menor desvio = maior uniformidade
    
    # 4. Score de qualidade composto
    coverage_score = min(coverage / 80 * 40, 40)  # Máximo 40 pontos
    vigor_score = (vigor_index / 255) * 30        # Máximo 30 pontos
    uniformity_score = (uniformity / 100) * 30   # Máximo 30 pontos
    
    quality_score = coverage_score + vigor_score + uniformity_score
    
    # Classificação de qualidade
    if quality_score >= 80:
        quality_class = "Excelente 🟢"
    elif quality_score >= 65:
        quality_class = "Boa 🟡"
    elif quality_score >= 45:
        quality_class = "Regular 🟠"
    else:
        quality_class = "Baixa 🔴"
    
    return {
        'quality_score': quality_score,
        'quality_class': quality_class,
        'coverage': coverage,
        'vigor_index': vigor_index,
        'uniformity': uniformity,
        'grass_type': grass_type
    }

# Analisar qualidade das gramíneas
if custom_analysis and grass_images:
    print("🌾 Analisando qualidade das pastagens...")
    
    grass_names = [
        "Brachiaria brizantha", "Brachiaria decumbens", "Brachiaria humidicola",
        "Panicum mombaça", "Panicum tanzânia", "Panicum massai",
        "Cynodon tifton", "Cynodon coast-cross"
    ]
    
    quality_results = []
    
    for analysis in custom_analysis:
        img_idx = analysis['image_idx']
        img = grass_images[img_idx]
        mask = analysis['mask']
        
        grass_type = grass_names[img_idx] if img_idx < len(grass_names) else "Gramínea mista"
        
        quality = analyze_pasture_quality(img, mask, grass_type)
        quality['image_idx'] = img_idx
        quality_results.append(quality)
        
        print(f"Gramínea {img_idx+1} ({grass_type}): {quality['quality_class']} (Score: {quality['quality_score']:.1f})")
    
    # Criar relatório visual de qualidade
    if quality_results:
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('📊 Relatório de Qualidade das Pastagens', fontsize=16, fontweight='bold')
        
        # 1. Scores de qualidade por gramínea
        grass_types = [q['grass_type'] for q in quality_results]
        quality_scores = [q['quality_score'] for q in quality_results]
        
        colors = ['green' if score >= 80 else 'yellow' if score >= 65 else 'orange' if score >= 45 else 'red' 
                 for score in quality_scores]
        
        axes[0, 0].bar(range(len(quality_scores)), quality_scores, color=colors)
        axes[0, 0].set_title('Score de Qualidade por Gramínea')
        axes[0, 0].set_ylabel('Score (0-100)')
        axes[0, 0].set_xticks(range(len(grass_types)))
        axes[0, 0].set_xticklabels([f"G{i+1}" for i in range(len(grass_types))], rotation=45)
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. Cobertura vs Vigor
        coverages = [q['coverage'] for q in quality_results]
        vigors = [q['vigor_index'] for q in quality_results]
        
        scatter = axes[0, 1].scatter(coverages, vigors, c=quality_scores, cmap='RdYlGn', s=100)
        axes[0, 1].set_title('Cobertura vs Índice de Vigor')
        axes[0, 1].set_xlabel('Cobertura (%)')
        axes[0, 1].set_ylabel('Índice de Vigor')
        plt.colorbar(scatter, ax=axes[0, 1], label='Score de Qualidade')
        
        # 3. Distribuição de qualidade
        quality_classes = [q['quality_class'].split()[0] for q in quality_results]  # Remover emoji
        class_counts = {}
        for qc in quality_classes:
            class_counts[qc] = class_counts.get(qc, 0) + 1
        
        axes[1, 0].pie(class_counts.values(), labels=class_counts.keys(), autopct='%1.1f%%')
        axes[1, 0].set_title('Distribuição de Classes de Qualidade')
        
        # 4. Métricas detalhadas
        metrics = ['Cobertura', 'Vigor', 'Uniformidade']
        avg_coverage = np.mean(coverages)
        avg_vigor = np.mean(vigors)
        avg_uniformity = np.mean([q['uniformity'] for q in quality_results])
        
        values = [avg_coverage, avg_vigor, avg_uniformity]
        
        bars = axes[1, 1].bar(metrics, values, color=['lightblue', 'lightgreen', 'lightcoral'])
        axes[1, 1].set_title('Métricas Médias')
        axes[1, 1].set_ylabel('Valor')
        
        # Adicionar valores nas barras
        for bar, value in zip(bars, values):
            axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                           f'{value:.1f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        # Resumo estatístico
        print("\n📈 Resumo Estatístico:")
        avg_quality = np.mean(quality_scores)
        print(f"Score médio de qualidade: {avg_quality:.1f}")
        print(f"Cobertura média: {avg_coverage:.1f}%")
        print(f"Vigor médio: {avg_vigor:.1f}")
        print(f"Uniformidade média: {avg_uniformity:.1f}")
        
        # Melhor e pior gramínea
        best_idx = np.argmax(quality_scores)
        worst_idx = np.argmin(quality_scores)
        
        print(f"\n🏆 Melhor gramínea: {quality_results[best_idx]['grass_type']} (Score: {quality_scores[best_idx]:.1f})")
        print(f"⚠️ Gramínea que precisa de atenção: {quality_results[worst_idx]['grass_type']} (Score: {quality_scores[worst_idx]:.1f})")

else:
    print("❌ Análise de qualidade não pode ser executada - dados insuficientes")

## 💾 Salvamento e Exportação

In [ ]:
# Salvar resultados da análise
import json
from datetime import datetime

print("💾 Salvando resultados da análise...")

# Preparar dados para exportação
export_data = {
    'timestamp': datetime.now().isoformat(),
    'analysis_type': 'Gramíneas de Pastos Brasileiros',
    'total_images_generated': len(grass_images) if grass_images else 0,
    'total_images_analyzed': len(custom_analysis) if custom_analysis else 0,
    'grass_types': [],
    'quality_analysis': [],
    'detection_summary': {}
}

# Adicionar dados de qualidade se disponíveis
if 'quality_results' in locals() and quality_results:
    for result in quality_results:
        export_data['quality_analysis'].append({
            'grass_type': result['grass_type'],
            'quality_score': float(result['quality_score']),
            'quality_class': result['quality_class'],
            'coverage_percentage': float(result['coverage']),
            'vigor_index': float(result['vigor_index']),
            'uniformity': float(result['uniformity'])
        })
    
    # Estatísticas resumidas
    quality_scores = [r['quality_score'] for r in quality_results]
    coverages = [r['coverage'] for r in quality_results]
    
    export_data['detection_summary'] = {
        'average_quality_score': float(np.mean(quality_scores)),
        'average_coverage': float(np.mean(coverages)),
        'best_grass_score': float(np.max(quality_scores)),
        'worst_grass_score': float(np.min(quality_scores)),
        'total_grass_types_analyzed': len(quality_results)
    }

# Adicionar informações sobre gramíneas analisadas
grass_types_info = [
    {'name': 'Brachiaria brizantha', 'characteristics': 'Alta produtividade, resistente à seca'},
    {'name': 'Brachiaria decumbens', 'characteristics': 'Boa adaptação, fácil estabelecimento'},
    {'name': 'Brachiaria humidicola', 'characteristics': 'Tolerante à umidade, solos pobres'},
    {'name': 'Panicum mombaça', 'characteristics': 'Alto valor nutritivo, grande porte'},
    {'name': 'Panicum tanzânia', 'characteristics': 'Boa digestibilidade, resistente'},
    {'name': 'Panicum massai', 'characteristics': 'Compacto, alta qualidade'},
    {'name': 'Cynodon tifton', 'characteristics': 'Textura fina, alta digestibilidade'},
    {'name': 'Cynodon coast-cross', 'characteristics': 'Resistente ao pisoteio'}
]

export_data['grass_types'] = grass_types_info

# Salvar arquivo JSON
filename = f"analise_gramineas_brasileiras_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, ensure_ascii=False, indent=2)

print(f"✅ Resultados salvos em: {filename}")

# Criar relatório em texto
report_filename = f"relatorio_gramineas_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
with open(report_filename, 'w', encoding='utf-8') as f:
    f.write("🌾 RELATÓRIO DE ANÁLISE DE GRAMÍNEAS DE PASTOS BRASILEIROS\n")
    f.write("=" * 60 + "\n\n")
    
    f.write(f"Data da análise: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n\n")
    
    f.write(f"Total de imagens geradas: {len(grass_images) if grass_images else 0}\n")
    f.write(f"Total de imagens analisadas: {len(custom_analysis) if custom_analysis else 0}\n\n")
    
    if 'quality_results' in locals() and quality_results:
        f.write("RESULTADOS DE QUALIDADE:\n")
        f.write("-" * 30 + "\n")
        
        for i, result in enumerate(quality_results):
            f.write(f"{i+1}. {result['grass_type']}\n")
            f.write(f"   Qualidade: {result['quality_class']}\n")
            f.write(f"   Score: {result['quality_score']:.1f}/100\n")
            f.write(f"   Cobertura: {result['coverage']:.1f}%\n")
            f.write(f"   Vigor: {result['vigor_index']:.1f}/255\n")
            f.write(f"   Uniformidade: {result['uniformity']:.1f}/100\n\n")
        
        # Estatísticas gerais
        avg_quality = np.mean([r['quality_score'] for r in quality_results])
        avg_coverage = np.mean([r['coverage'] for r in quality_results])
        
        f.write("ESTATÍSTICAS GERAIS:\n")
        f.write("-" * 20 + "\n")
        f.write(f"Score médio de qualidade: {avg_quality:.1f}\n")
        f.write(f"Cobertura média: {avg_coverage:.1f}%\n")
    
    f.write("\nGRAMÍNEAS ANALISADAS:\n")
    f.write("-" * 25 + "\n")
    for grass in grass_types_info:
        f.write(f"• {grass['name']}: {grass['characteristics']}\n")

print(f"✅ Relatório salvo em: {report_filename}")

# Informações finais
print("\n📊 Informações de Performance:")
if torch.cuda.is_available():
    print(f"Memória GPU alocada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Memória GPU máxima: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

print("\n✅ Análise de gramíneas brasileiras concluída com sucesso!")
print("\n🎯 Resumo dos Resultados:")
if grass_images:
    print(f"• {len(grass_images)} imagens sintéticas de gramíneas geradas")
if custom_analysis:
    print(f"• {len(custom_analysis)} imagens analisadas com segmentação personalizada")
if 'quality_results' in locals() and quality_results:
    avg_score = np.mean([r['quality_score'] for r in quality_results])
    print(f"• Score médio de qualidade: {avg_score:.1f}/100")
    print(f"• {len(quality_results)} gramíneas avaliadas")

print("\n🌾 Próximos passos sugeridos:")
print("• Expandir dataset com mais variedades de gramíneas")
print("• Implementar detecção de doenças e pragas")
print("• Desenvolver sistema de monitoramento temporal")
print("• Integrar dados de solo e clima")